# 구글 검색 결과 수집

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bts
import pandas as pd
import time, os, datetime

### 리모트 드라이버 설정

In [ ]:
# 크롬 드라이버가 저장된 경로를 지정합니다.
# [주의] 반드시 절대경로로 지정해야 합니다.
service = webdriver.chrome.service.Service(
    executable_path = '/Users/drkevin/Selenium/chromedriver'
)

In [ ]:
# 크롬 드라이버 옵션을 설정합니다.
options = webdriver.chrome.options.Options()
options.add_argument('window-size=1280,1960')

In [ ]:
# 리모트 드라이버를 설정합니다.
remote = webdriver.Chrome(service = service, options = options)

### 웹 페이지 열기

In [ ]:
# 구글 메인 페이지를 엽니다.
remote.get(url = 'https://www.google.com')

In [ ]:
# 암묵적으로 최대 3초간 기다립니다.
remote.implicitly_wait(time_to_wait = 3)

In [ ]:
# 무조건 3초간 기다립니다.
time.sleep(3)

### 검색어 입력

In [ ]:
# 검색창을 지정합니다.
# [참고] Selenium은 XPath를 사용하는 것이 좋습니다.
query = remote.find_element(
    by = 'xpath', 
    value = '//form/div[1]/div[1]/div[1]/div/div[2]/input'
)

In [ ]:
# 검색창에 검색어를 입력하고, 엔터를 누릅니다.
query.send_keys('빅데이터; 인공지능', Keys.ENTER)

### 웹 페이지 이동

In [ ]:
# 현재 페이지의 URL을 출력합니다.
remote.current_url

In [ ]:
# 현재 페이지를 새로고침합니다.
remote.refresh()

In [ ]:
# 이전 페이지로 이동합니다.(뒤로 가기)
remote.back()

In [ ]:
# 다음 페이지로 이동합니다.(앞으로 가기)
remote.forward()

### HTML로 변환

In [ ]:
# 현재 웹 페이지의 HTML을 읽고, html을 생성합니다.
html = remote.page_source

In [ ]:
# html을 파싱하여 soup을 생성합니다.
soup = bts(markup = html, features = 'html.parser')

In [ ]:
# 검색 결과를 포함하는 HTML 요소를 선택하고, items를 생성합니다.
items = soup.select(selector = 'div.yuRUbf')

In [ ]:
# items의 원소 개수를 확인합니다.
len(items)

### 웹 페이지 데이터 수집

In [ ]:
# 제목을 수집합니다.
head = [item.select(selector = 'h3.LC20lb')[0].text for item in items]

In [ ]:
# 링크를 수집합니다.
link = [item.select(selector = 'a')[0]['href'] for item in items]

In [ ]:
# 제목과 링크로 데이터프레임을 생성합니다.
df = pd.DataFrame(data = {'제목': head, '링크': link})

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 스크롤 이동

In [ ]:
# 웹 페이지를 아래로 500px 이동합니다.
remote.execute_script(script = 'window.scrollBy(0, 500)')

In [ ]:
# 웹 페이지를 위로 300px 이동합니다.
remote.execute_script(script = 'window.scrollBy(0, -300)')

In [ ]:
# 웹 페이지 맨 위로 이동합니다.
remote.execute_script(script = 'window.scrollTo(0, 0)')

In [ ]:
# 웹 페이지 맨 아래로 이동합니다.
remote.execute_script(script = 'window.scrollTo(0, document.body.scrollHeight)')

### 페이지 네비게이션

In [ ]:
# 다음 페이지 버튼을 클릭합니다.
node = '//*[@id="pnnext"]/span[2]'
remote.find_element(by = 'xpath', value = node).click()

In [ ]:
# 이전 페이지 버튼을 클릭합니다.
node = '//*[@id="pnprev"]/span[2]'
remote.find_element(by = 'xpath', value = node).click()

### xlsx 파일로 저장

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# xlsx 파일명을 설정합니다.
today = datetime.datetime.now().strftime('%Y%m%d')
fileName = f'Google_Search_{today}.xlsx'

In [ ]:
# df를 xlsx 파일로 저장합니다.
df.to_excel(excel_writer = fileName, index = None)

### 리모트 드라이버 닫기

In [ ]:
# 현재 웹 브라우저를 닫습니다.
remote.close()

In [ ]:
# 크롬 드라이버를 종료합니다.
remote.quit()

## End of Document